In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import cPickle as pickle
import multiprocessing
import scipy.special
import numpy as np
import numpy.fft as fft
import time
import os
import sys
sys.path.insert(1,'/Users/zyzdiana/GitHub/AC297r-Volume-Registration/code')

In [3]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import plotly.graph_objs as go

In [4]:
from conversion import rotation_matrix_fromq,rotation_matrix_from_params,rotation_matrix_to_q,angles_from_q
from utils import to_radian,res_to_rad,ax_to_w,rep_to_angle

In [5]:
# Get All Axes of Rotations
path = '/Users/zyzdiana/Dropbox/THESIS/July_26_2016_navs/'
resolutions = ['6_4mm', '8mm', '10mm']
ranges = ['0_5_to_2_5','3_0_to_5_0','2_5_to_12_5','15_to_25']
files = os.listdir(path)
axes_dict = {}
for f in files[1:]:
    rot_ax = f.split('_')[2]
    trans_ax = f.split('_')[10]
    if not axes_dict.has_key(rot_ax):
        axes_dict[rot_ax] = trans_ax
print axes_dict
keys = axes_dict.keys()

rot_axes = ['xz', 'y', 'yz', 'xy', 'x', 'z']
ax_to_idx = {}
for rot_ax in rot_axes:
    ax_to_idx[rot_ax] = rot_axes.index(rot_ax)
print ax_to_idx

{'xz': 'xy', 'y': 'z', 'yz': 'xy', 'xy': 'xy', 'x': 'z', 'z': 'z'}
{'xy': 3, 'xz': 0, 'yz': 2, 'y': 1, 'x': 4, 'z': 5}


In [6]:
def Max_displacement(dRM, R_axis, dt):
    rad = 100
    trans_R = rad*np.sqrt(3-np.trace(dRM))
    E_max = np.sqrt(trans_R**2 + 2*trans_R*np.linalg.norm(dt-(dt.T.dot(R_axis))*R_axis) + (dt.T).dot(dt))
    return trans_R, E_max

In [7]:
def RMS_Rotation(dRM):
    return np.arccos((np.trace(dRM)-1)/2.)*180/np.pi

def RMS_Translation(dt):
    t = dt
    return np.sqrt((t.T).dot(t))
def RMS_Translation_R(dRM):
    rad = 100
    A = dRM - np.eye(3)
    return np.sqrt(0.2*rad**2*np.trace((A.T).dot(A)))

def RMS(dRM, dt, res_f):
    rad = 100
    A = dRM - np.eye(3)
    return np.sqrt(0.2*rad**2*np.trace((A.T).dot(A))+ (dt.T).dot(dt))
    
from conversion import rotation_matrix_fromq, rotation_matrix_from_params
def get_true_params(rot_angle,rot_ax):
    wy,wx,wz = ax_to_w(rot_ax)
    wy_t,wx_t,wz_t = ax_to_w(axes_dict[rot_ax])
    true_RM = rotation_matrix_fromq(rot_angle[0],wx,wy,wz)
    trans_y, trans_x, trans_z = wy_t*rot_angle[1],wx_t*rot_angle[1], wz_t*rot_angle[1]
    return [trans_x, trans_y, trans_z], true_RM

def get_params(params, res):
    RM = rotation_matrix_from_params(-params[3:])
    return  params[:3]*res, RM

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v/norm

In [8]:
keys = ['xz', 'y', 'yz', 'xy', 'x', 'z']

def compute_RMS_from_C_output(output_path, output_filename):
    results = np.loadtxt(os.path.join(output_path,output_filename))
    splits = output_filename.split('_')
    res = splits[0][:-2]
    res_f = float('.'.join(res.split('_')))
    interp = splits[1]
    rot_ax = splits[2]
    trans_ax = splits[10]
    rang = '_'.join(splits[4:9])
    
    true_params = []
    Max_dis = []
    Max_dis_R = []
    RMS_ls = []
    RMS_rotation_ls = []
    RMS_translation_ls = []
    RMS_trans_R = []
    counter_ls = []    
    
    ref = rep_to_angle(0,rang)
    
    idx = 0
    for rep in xrange(1,36):
        rot_angle = rep_to_angle(rep,rang)
        true_params.append(rot_angle)
        true_t, true_RM = get_true_params(rot_angle,rot_ax)
        rad = res_to_rad(res)
        
        for algo in xrange(8):
            counter = results[idx,1]
            Ps = results[idx,2:]
            R_axis = normalize(Ps[3:])
            t , RM = get_params(Ps, res_f)
            dt = true_t + t
            dR = RM.dot(true_RM.T)
            max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
            Max_dis.append(E_Max)
            Max_dis_R.append(max_trans_R)
            RMS_ls.append(RMS(dR, dt, res_f))
            
            if(RMS(dR, dt, res_f) > 1):
                print idx, rot_ax, rang, rep
            if(RMS(dR, dt, res_f) > E_Max):
                print rot_ax, rang, rep, rot_angle
                print RMS(dR, dt, res_f), E_Max

            RMS_rotation_ls.append(RMS_Rotation(dR))
            RMS_translation_ls.append(RMS_Translation(dt))
            RMS_trans_R.append(RMS_Translation_R(dR))
            counter_ls.append(counter)
            idx += 1
    return [true_params, 
            RMS_ls, RMS_rotation_ls, RMS_translation_ls, 
            Max_dis, Max_dis_R, RMS_trans_R, counter_ls]  


def compute_RMS(output_path, interp, algo, num_algo = 6, range_start = 0, range_end = 2, keys = keys, axes_dict = axes_dict):
    true_params = []
    Max_dis_6_4 = []
    Max_dis_R_6_4 = []
    RMS_ls_6_4 = []
    RMS_rotation_ls_6_4 = []
    RMS_translation_ls_6_4 = []
    RMS_trans_R_6_4 = []
    counter_6_4 = []
    
    Max_dis_8 = []
    Max_dis_R_8 = []
    RMS_ls_8 = []
    RMS_rotation_ls_8 = []
    RMS_translation_ls_8 = []
    RMS_trans_R_8 = []
    counter_8 = []
    
    Max_dis_10 = []
    Max_dis_R_10 = []
    RMS_ls_10 = []
    RMS_rotation_ls_10 = []
    RMS_translation_ls_10 = []
    RMS_trans_R_10 = []
    counter_10 = []
    for ix, rot_ax in enumerate(keys):
        for rang in ranges[range_start:range_end]:
            ref = rep_to_angle(0,rang)
            for rep in xrange(1,36):
                rot_angle = rep_to_angle(rep,rang)
                true_params.append(rot_angle)
                true_t, true_RM = get_true_params(rot_angle,rot_ax)

                res = '6_4mm'
                res_f =  6.4
                rad = res_to_rad(res)
                output_file = '%s_%s_rot_%s_deg_%s_trans_%s.txt' % (res, rot_ax, rang, axes_dict[rot_ax], interp)
                results = np.loadtxt(os.path.join(output_path,output_file))
                idx = algo+(rep-1)*num_algo
                counter = results[idx,1]
                Ps = results[idx,2:]
                R_axis = normalize(Ps[3:])
                t , RM = get_params(Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_6_4.append(E_Max)
                Max_dis_R_6_4.append(max_trans_R)
                RMS_ls_6_4.append(RMS(dR, dt, res_f))
#                 if(RMS(dR, dt, res_f) > 1):
#                     print ix, rot_ax, rang, rep
#                 if(RMS(dR, dt, res_f) > E_Max):
#                     print rot_ax, rang, rep, rot_angle
#                     print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_6_4.append(RMS_Rotation(dR))
                RMS_translation_ls_6_4.append(RMS_Translation(dt))
                RMS_trans_R_6_4.append(RMS_Translation_R(dR))
                counter_6_4.append(counter)

                res = '8mm'
                res_f =  8.0
                rad = res_to_rad(res)
                output_file = '%s_iso_%s_rot_%s_deg_%s_trans_%s.txt' % (res, rot_ax, rang, axes_dict[rot_ax],interp)
                results = np.loadtxt(os.path.join(output_path,output_file))
                #for algo in xrange(8):
                idx = algo+(rep-1)*num_algo
                counter = results[idx,1]
                Ps = results[idx,2:]
                R_axis = normalize(Ps[3:])
                t , RM = get_params(Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_8.append(E_Max)
                Max_dis_R_8.append(max_trans_R)
                RMS_ls_8.append(RMS(dR, dt, res_f))
#                 if(RMS(dR, dt, res_f) > 1):
#                     print ix, rot_ax, rang, rep
#                 if(RMS(dR, dt, res_f) > E_Max):
#                     print rot_ax, rang, rep, rot_angle
#                     print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_8.append(RMS_Rotation(dR))
                RMS_translation_ls_8.append(RMS_Translation(dt))
                RMS_trans_R_8.append(RMS_Translation_R(dR))
                counter_8.append(counter)

                res = '10mm'
                res_f =  10.0
                rad = res_to_rad(res)
                output_file = '%s_iso_%s_rot_%s_deg_%s_trans_%s.txt' % (res, rot_ax, rang, axes_dict[rot_ax], interp) 
                results = np.loadtxt(os.path.join(output_path,output_file))
                #for algo in xrange(8):
                idx = algo+(rep-1)*num_algo   
                counter = results[idx,1]
                Ps = results[idx,2:]
                R_axis = normalize(Ps[3:])
                t , RM = get_params(Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_10.append(E_Max)
                Max_dis_R_10.append(max_trans_R)
                RMS_ls_10.append(RMS(dR, dt, res_f))
#                 if(RMS(dR, dt, res_f) > 1):
#                     print ix, rot_ax, rang, rep
#                 if(RMS(dR, dt, res_f) > E_Max):
#                     print rot_ax, rang, rep, rot_angle
#                     print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_10.append(RMS_Rotation(dR))
                RMS_translation_ls_10.append(RMS_Translation(dt))
                RMS_trans_R_10.append(RMS_Translation_R(dR))
                counter_10.append(counter)
                
    return [true_params,
            RMS_ls_6_4,RMS_ls_8,RMS_ls_10,
            RMS_rotation_ls_6_4,RMS_rotation_ls_8,RMS_rotation_ls_10, 
            RMS_translation_ls_6_4,RMS_translation_ls_8,RMS_translation_ls_10,
            Max_dis_6_4,Max_dis_8,Max_dis_10,
            Max_dis_R_6_4,Max_dis_R_8,Max_dis_R_10,
            RMS_trans_R_6_4, RMS_trans_R_8, RMS_trans_R_10,
            counter_6_4, counter_8, counter_10]

## Get all Bspline results for every subject

In [9]:
output_path = '/Users/zyzdiana/Desktop/C_output_oribi/'
interps = ['linear','cubic','cubicBSpline']
for interp in interps:
    for ix, rot_ax in enumerate(keys):
        for rang in ranges[:2]:
            ref = rep_to_angle(0,rang)
            for res in resolutions:
                float_res = float('.'.join(res[:-2].split('_')))
                width = res_to_rad(res)*2
                if res == '6_4mm':
                    output_file = '%s_%s_rot_%s_deg_%s_trans_%s.txt' % (res, rot_ax, rang, axes_dict[rot_ax], interp)
                else:
                    output_file = '%s_iso_%s_rot_%s_deg_%s_trans_%s.txt' % (res, rot_ax, rang, axes_dict[rot_ax], interp)
                #print output_file

In [10]:
dates = ['Oct_13_navs', 'June_8_2016_navs', 'July_26_2016_Navs', 'July_15_2016_Navs']
interps = ['linear','cubic','cubicBSpline']
data_ls_all = []
output_path = '/Users/zyzdiana/Desktop/C_output_oribi/'
for algo in xrange(6):
    print 'algo',algo
    data_ls = []
    for interp in interps:
        print interp,
        for date in dates:
            print date,
            path = os.path.join(output_path, date)
            data_ls.append(compute_RMS(path, interp, algo, 6, range_start = 0, range_end = 2))
        print
    data_ls_all.append(data_ls)

algo 0
linear Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 1
linear Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 2
linear Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 3
linear Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 4
linear Oct_13_navs June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic Oct

In [11]:
pickle.dump(data_ls_all,open('data_ls_all.p','wb'))

In [10]:
data_ls_all = pickle.load(open('data_ls_all.p','rb'))

In [77]:
dates = ['June_8_2016_navs', 'July_26_2016_Navs', 'July_15_2016_Navs']
interps = ['linear','cubic','cubicBSpline']
large_rot_data_ls_all = []
output_path = '/Users/zyzdiana/Desktop/C_output_oribi/'
for algo in xrange(6):
    print 'algo',algo
    data_ls = []
    for interp in interps:
        print interp,
        for date in dates:
            print date,
            path = os.path.join(output_path, date)
            data_ls.append(compute_RMS(path, interp, algo, 6, range_start = 2, range_end = 4, keys = ['x']))
        print
    large_rot_data_ls_all.append(data_ls)

algo 0
linear June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 1
linear June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 2
linear June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 3
linear June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 4
linear June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubic June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
cubicBSpline June_8_2016_navs July_26_2016_Navs July_15_2016_Navs
algo 5
linear June_8_2016_navs July_26_2

In [78]:
pickle.dump(large_rot_data_ls_all,open('large_rot_data_ls_all.p','wb'))

In [11]:
large_rot_data_ls_all = pickle.load(open('large_rot_data_ls_all.p','rb'))

## Make Dataframe

In [12]:
import pandas as pd
labels = ['10mm<br>Trilinear','10mm<br>Tricubic','10mm<br>Cubic<br>B-spline',
          '8mm<br>Trilinear','8mm<br>Tricubic','8mm<br>Cubic<br>B-spline',
          '6.4mm<br>Trilinear','6.4mm<br>Tricubic','6.4mm\nCubic<br>B-spline']

columns = ['Label','RMS','RMS_Rotation','RMS_Translation','Max_Dis','Max_Dis_R','RMS_trans_R','Iterations']

In [13]:
num_subject = 3
Labels_ls = []
for l in labels:
    for subj in xrange(1,num_subject+1):
        for j in xrange(420):
            Labels_ls.append(l+'<br>All 3 subjects')
print len(Labels_ls)

num_subject = 2
Labels_ls_large_rot = []
for l in labels:
    for subj in xrange(1,num_subject+1):
        for j in xrange(70):
            Labels_ls_large_rot.append(l+'<br>All 2 subjects')
print len(Labels_ls_large_rot)

Pineapple_label = []
for l in labels:
    for j in xrange(420):
        Pineapple_label.append(l+'<br>Pineapple')
print len(Pineapple_label)

Pineapple_label_large_rot = []
for l in labels:
    for j in xrange(70):
        Pineapple_label_large_rot.append(l+'<br>Pineapple')
print len(Pineapple_label_large_rot)

11340
1260
3780
630


In [14]:
def get_subjects_df(data_ls, Labels_ls = Labels_ls, num_subject = 3):
    subjects_df = pd.DataFrame(columns = columns)
    subjects_df[columns[0]] = Labels_ls
    for idx, col in enumerate(columns[1:]):
        col_vals = []
        for res in [3,2,1]:
            for interp in xrange(3):
                for subj in xrange(num_subject):
                    col_vals += data_ls[interp*num_subject+subj][res+(idx*3)]
        subjects_df[col] = col_vals
    return subjects_df

def get_pineapple_df(data_ls, Pineapple_label = Pineapple_label, pineapple_idx = 3, num_subject = 4):
    pineapple_df = pd.DataFrame(columns = columns)
    pineapple_df[columns[0]] = Pineapple_label
    for idx, col in enumerate(columns[1:]):
        col_vals = []
        for res in [3,2,1]:
            for interp in xrange(3):
                col_vals += data_ls[interp*num_subject+pineapple_idx][res+(idx*3)]
        pineapple_df[col] = col_vals
    return pineapple_df

In [15]:
df = get_pineapple_df(data_ls_all[0])
df.head()

,Label,RMS,RMS_Rotation,RMS_Translation,Max_Dis,Max_Dis_R,RMS_trans_R,Iterations
0,10mm<br>Trilinear<br>Pineapple,0.176684,0.106972,0.131432,0.314493,0.186701,0.118080,2
1,10mm<br>Trilinear<br>Pineapple,0.263906,0.226214,0.085404,0.472513,0.394818,0.249705,2
2,10mm<br>Trilinear<br>Pineapple,0.369944,0.333048,0.041297,0.606074,0.581277,0.367632,2
3,10mm<br>Trilinear<br>Pineapple,0.495513,0.446724,0.048722,0.811732,0.779679,0.493112,1
4,10mm<br>Trilinear<br>Pineapple,0.628021,0.565817,0.065744,1.024416,0.987533,0.624571,1


## Violin Plot

In [16]:
from plotly.offline import plot
import plotly.graph_objs as go

In [17]:
for algo in xrange(6):
    fig = FF.create_violin(get_subjects_df(data_ls_all[algo]), data_header='RMS', group_header='Label', 
                           height=800, width=2800, title='RMS Error (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/RMS_algo%s.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/RMS_algo%s.svg' % (algo+1), auto_open = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1)

In [19]:
for algo in xrange(6):
    fig = FF.create_violin(get_pineapple_df(data_ls_all[algo]), data_header='RMS', group_header='Label', 
                           height=800, width=2800, title='RMS Error (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Pinapple_RMS_algo%s.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Pinapple_RMS_algo%s.html' % (algo+1), auto_open = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1)

In [20]:
for algo in xrange(6):
    fig = FF.create_violin(get_subjects_df(data_ls_all[algo]), data_header='Max_Dis', group_header='Label', 
                           height=800, width=2800, title='Max Displacement (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Max_algo%s.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Max_algo%s.html' % (algo+1), auto_open = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1)

In [21]:
for algo in xrange(6):
    fig = FF.create_violin(get_pineapple_df(data_ls_all[algo]), data_header='Max_Dis', group_header='Label', 
                           height=800, width=2800, title='Max Displacement (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Pineapple_Max_algo%s.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Pineapple_Max_algo%s.html' % (algo+1), auto_open = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1)

In [23]:
for algo in xrange(6):
    fig = FF.create_violin(get_subjects_df(data_ls_all[algo]), data_header='Iterations', group_header='Label', 
                           height=800, width=2800, title='Iterations (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Iterations_algo%s.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Iterations_algo%s.html' % (algo+1), auto_open = False)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]  [ (1,3) x3,y1 ]  [ (1,4) x4,y1 ]  [ (1,5) x5,y1 ]  [ (1,6) x6,y1 ]  [ (1,7) x7,y1 ]  [ (1,8) x8,y1 ]  [ (1,9) x9,y1 ]

This is the format of your plot grid:
[ (1,1)

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_pineapple_df(data_ls_all[algo]), data_header='Iterations', group_header='Label', 
                           height=800, width=2800, title='Iterations (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Pinapple_Iterations_algo%s.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Pinapple_Iterations_algo%s.html' % (algo+1), auto_open = False)

## Large Rotations

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_subjects_df(larget_rot_data_ls_all[algo], Labels_ls_large_rot, 2), data_header='RMS', 
                           group_header='Label', height=800, width=2800, title='RMS Error (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/RMS_algo%s_large_rot.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/RMS_algo%s_large_rot.html' % (algo+1), auto_open = False)

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_pineapple_df(large_rot_data_ls_all[0], Pineapple_label_large_rot, 2, 3), data_header='RMS', 
                           group_header='Label', height=800, width=2800, title='RMS Error (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Pineapple_RMS_algo%s_large_rot.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Pineapple_RMS_algo%s_large_rot.html' % (algo+1), auto_open = False)

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_subjects_df(larget_rot_data_ls_all[algo], Labels_ls_large_rot, 2), data_header='Max_Dis', 
                           group_header='Label', height=800, width=2800, title='Max Displacement Error (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Max_algo%s_large_rot.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Max_algo%s_large_rot.html' % (algo+1), auto_open = False)

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_pineapple_df(large_rot_data_ls_all[0], Pineapple_label_large_rot, 2, 3), data_header='Max_Dis', 
                           group_header='Label', height=800, width=2800, title='Max Displacement Error (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Pineapple_Max_algo%s_large_rot.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Pineapple_Max_algo%s_large_rot.html' % (algo+1), auto_open = False)

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_subjects_df(larget_rot_data_ls_all[algo], Labels_ls_large_rot, 2), data_header='Iterations', 
                           group_header='Label', height=800, width=2800, title='Iterations (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Iterations_algo%s_large_rot.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Iterations_algo%s_large_rot.html' % (algo+1), auto_open = False)

In [ ]:
for algo in xrange(6):
    fig = FF.create_violin(get_pineapple_df(large_rot_data_ls_all[0], Pineapple_label_large_rot, 2, 3), data_header='Iterations', 
                           group_header='Label', height=800, width=2800, title='Iterations (Algo %s)' % (algo+1))
    py.image.save_as(fig, '/Users/zyzdiana/Desktop/htmls/Pineapple_Iterations_algo%s_large_rot.svg' % (algo+1))
    #plot(fig, filename='/Users/zyzdiana/Desktop/htmls/Pineapple_Iterations_algo%s_large_rot.html' % (algo+1), auto_open = False)

### Plot in Notebook

In [ ]:
fig = FF.create_violin(get_subjects_df(data_ls_all[0]), data_header='RMS', group_header='Label', 
                       height=800, width=2800, title='RMS Error (Algo 1)')
print py.iplot(fig)

In [ ]:
fig = FF.create_violin(get_subjects_df(data_ls_all[1]), data_header='RMS', group_header='Label', 
                       height=800, width=2800, title='RMS Error (Algo 2)')
py.iplot(fig)

In [ ]:
fig = FF.create_violin(get_subjects_df(data_ls_all[2]), data_header='RMS', group_header='Label', 
                       height=800, width=2800, title='RMS Error (Algo 3)')
py.iplot(fig)

In [ ]:
fig = FF.create_violin(get_subjects_df(data_ls_all[3]), data_header='RMS', group_header='Label', 
                       height=800, width=2800, title='RMS Error (Algo 4)')
py.iplot(fig)

In [ ]:
fig = FF.create_violin(get_subjects_df(data_ls_all[4]), data_header='RMS', group_header='Label', 
                       height=800, width=2800, title='RMS Error (Algo 5)')
py.iplot(fig)

In [ ]:
fig = FF.create_violin(get_subjects_df(data_ls_all[5]), data_header='RMS', group_header='Label', 
                       height=800, width=2800, title='RMS Error (Algo 6)')
py.iplot(fig)

# Scatter Plots

In [85]:
c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, 6)]
colors = []
for color in c:
    colors += [color]*70
    
xx = np.ones(420)

rot_axes = ['x','y','z','xy','yz','xz']
ax_labels = []
for ax in rot_axes:
    ax_labels += [ax]*70

labels = ['10mm\nTrilinear','10mm\nTricubic','10mm\nCubic\nB-spline',
          '8mm\nTrilinear','8mm\nTricubic','8mm\nCubic\nB-spline',
          '6.4mm\nTrilinear','6.4mm\nTricubic','6.4mm\nCubic\nB-spline']

def plotly_scatter(data_ls, idx_ls, title, ylabel):
    l = []
    index = 1

    for idx in idx_ls:
        for ii, ls in enumerate(data_ls):
            subj_idx = (ii+1) % 4
            trace0= go.Scatter(
                x= ((xx * index) - 0.3) + (0.2*subj_idx),
                y= ls[idx],
                mode= 'markers',
                marker= dict(size= 14,
                            line= dict(width=1),
                            color= colors,#c[index-1],
                            opacity= 0.3
                           ),
                name= 'subject %s' % subj_idx,
                text= ax_labels) # The hover text goes here... 
            l.append(trace0);
            if(subj_idx) == 0:
                index += 1

    layout= go.Layout(
        title= title,
        hovermode= 'closest',
        xaxis= dict(
            title= '',
            ticklen= 5,
            zeroline= False,
            gridwidth= 2,
            ticktext = labels,
            tickvals = [1,2,3,4,5,6,7,8,9]
        ),
        yaxis=dict(
            title= ylabel,
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend= False
    )
    fig= go.Figure(data=l, layout=layout)
    return fig

In [ ]:
fig = plotly_scatter(data_ls_all[2],[3,2,1], 'RMS error', 'RMS')
py.iplot(fig)